In [1]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.layers import Input, Dense, concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import keras_tuner as kt
import tensorflow as tf # Đảm bảo TensorFlow được import
import os
import cv2
import mediapipe as mp
import keypoint_extract as md # Import keypoint_extract

2025-07-18 11:21:32.987719: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-18 11:21:33.012757: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# --- Cấu hình đường dẫn và hằng số ---
DATA_PATH = 'data_separate.pickle'
LABEL_BINARIZER_PATH = 'label_binarizer.pkl'
TEST_IMAGE_FOLDER = 'VSL_DATA_2' # Thư mục chứa ảnh test riêng biệt

In [3]:
# Kích thước chuẩn cho từng loại đặc trưng (phải khớp với create_dataset.py)
MAX_LEN_LEFT_HAND = 21 * 3 # 63
MAX_LEN_RIGHT_HAND = 21 * 3 # 63
MAX_LEN_POSE_BODY = 33 * 3 # 99

# --- 1. Tải dữ liệu huấn luyện và validation ---
try:
    with open(DATA_PATH, 'rb') as f:
        data_dict = pickle.load(f)
except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file {DATA_PATH}. Hãy chạy create_dataset.py trước.")
    exit()

X_left_hand = np.array(data_dict['data_left_hand'], dtype=np.float32)
X_right_hand = np.array(data_dict['data_right_hand'], dtype=np.float32)
X_pose_body = np.array(data_dict['data_pose_body'], dtype=np.float32)
labels = data_dict['labels']


# 2. Tiền xử lý nhãn (Labels)

In [4]:
# Chuyển đổi nhãn từ chuỗi sang số nguyên, sau đó one-hot encoding
lb = LabelBinarizer()
y_encoded = lb.fit_transform(labels)
num_classes = len(lb.classes_)
class_names = lb.classes_  # Lưu lại để sử dụng cho đánh giá

# Ghép nối các đặc trưng thành một vector đầu vào duy nhất  
X_combined = np.concatenate((X_left_hand, X_right_hand, X_pose_body), axis=1)

# Chia dữ liệu
X_train, X_val, y_train, y_val = train_test_split(
    X_combined, y_encoded, test_size=0.2, random_state=42, stratify=labels)

print(f"Kích thước tập huấn luyện: {len(X_train)}")
print(f"Kích thước tập Validation: {len(X_val)}")


Kích thước tập huấn luyện: 54400
Kích thước tập Validation: 13600


In [5]:
# Xây dựng mô hình ANN đơn giản
def build_model(hp):
    # Đầu vào là vector đặc trưng đã được ghép nối
    input_layer = Input(shape=(X_combined.shape[1],))
    
    # Layer 1
    x = Dense(
        hp.Int('units_1', min_value=128, max_value=512, step=64),
        activation='relu'
    )(input_layer)
    x = Dropout(hp.Float('dropout_1', min_value=0.2, max_value=0.5, step=0.1))(x)
    
    # Layer 2
    x = Dense(
        hp.Int('units_2', min_value=64, max_value=256, step=32),
        activation='relu'
    )(x)
    x = Dropout(hp.Float('dropout_2', min_value=0.2, max_value=0.5, step=0.1))(x)
    
    # Layer 3
    x = Dense(
        hp.Int('units_3', min_value=32, max_value=128, step=32),
        activation='relu'
    )(x)
    
    # Lớp đầu ra
    output = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=input_layer, outputs=output)
    
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.compile(optimizer=Adam(learning_rate=hp_learning_rate),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])
    return model


In [6]:
# Khởi tạo và chạy Tuner
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=50,
    factor=3,
    directory='hyperband_dir_ann',
    project_name='sign_language_ann_tuning',
    overwrite=True
)


In [7]:
print("\nBắt đầu tìm kiếm siêu tham số tốt nhất...")
tuner.search(
    X_train,
    y_train,
    epochs=50,
    batch_size=64,
    validation_data=(X_val, y_val),
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)]
)


Trial 90 Complete [00h 00m 47s]
val_accuracy: 0.5913235545158386

Best val_accuracy So Far: 0.9569852948188782
Total elapsed time: 00h 25m 54s


In [8]:
# Chuyển đổi nhãn từ chuỗi sang số nguyên, sau đó one-hot encoding
lb = LabelBinarizer()
y_encoded = lb.fit_transform(labels)
num_classes = len(lb.classes_)
class_names = lb.classes_ # Lưu lại để sử dụng cho đánh giá

print(f"Tổng số mẫu dữ liệu: {len(labels)}")
print(f"Tổng số lớp (từ): {num_classes}")

Tổng số mẫu dữ liệu: 68000
Tổng số lớp (từ): 68


In [9]:
# --- 6. Lưu LabelBinarizer ---
try:
    with open(LABEL_BINARIZER_PATH, 'wb') as f:
        pickle.dump(lb, f)
    print(f"\nĐã lưu LabelBinarizer vào file {LABEL_BINARIZER_PATH}")
except Exception as e:
    print(f"Lỗi khi lưu LabelBinarizer: {e}")



Đã lưu LabelBinarizer vào file label_binarizer.pkl


In [17]:
# --- 4. Chuẩn bị dữ liệu ảnh test từ VSL_DATA_2 để đánh giá các mô hình của Tuner ---
print(f"\n--- Chuẩn bị dữ liệu ảnh test từ {TEST_IMAGE_FOLDER} để đánh giá các mô hình của Tuner ---\n")

mp_hands_test = mp.solutions.hands
mp_pose_test = mp.solutions.pose
hands_test = mp_hands_test.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.5)
pose_test = mp_pose_test.Pose(static_image_mode=True, min_detection_confidence=0.5)

# Hàm extract đặc trưng từ ảnh cho mô hình đa đầu vào (Tương tự Test_ảnh.py)
def extract_features_from_image_multi_input(img_path):
    img = cv2.imread(img_path)
    if img is None:
        return None, None, None

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    left_hand_landmarks, right_hand_landmarks = md.extract_hand_landmark(img_rgb, hands_test)
    list_pose = md.extract_body_landmark(img_rgb, pose_test)

    flat_left_hand = [coord for lm in left_hand_landmarks for coord in lm]
    flat_right_hand = [coord for lm in right_hand_landmarks for coord in lm]
    flat_pose_body = [coord for lm in list_pose for coord in lm]

    if len(flat_left_hand) < MAX_LEN_LEFT_HAND:
        flat_left_hand.extend([0.0] * (MAX_LEN_LEFT_HAND - len(flat_left_hand)))
    elif len(flat_left_hand) > MAX_LEN_LEFT_HAND:
        flat_left_hand = flat_left_hand[:MAX_LEN_LEFT_HAND]

    if len(flat_right_hand) < MAX_LEN_RIGHT_HAND:
        flat_right_hand.extend([0.0] * (MAX_LEN_RIGHT_HAND - len(flat_right_hand)))
    elif len(flat_right_hand) > MAX_LEN_RIGHT_HAND:
        flat_right_hand = flat_right_hand[:MAX_LEN_RIGHT_HAND]

    if len(flat_pose_body) < MAX_LEN_POSE_BODY:
        flat_pose_body.extend([0.0] * (MAX_LEN_POSE_BODY - len(flat_pose_body)))
    elif len(flat_pose_body) > MAX_LEN_POSE_BODY:
        flat_pose_body = flat_pose_body[:MAX_LEN_POSE_BODY]

    np_left_hand = np.array(flat_left_hand, dtype=np.float32).reshape(1, -1)
    np_right_hand = np.array(flat_right_hand, dtype=np.float32).reshape(1, -1)
    np_pose_body = np.array(flat_pose_body, dtype=np.float32).reshape(1, -1)

    return np_left_hand, np_right_hand, np_pose_body

# Lấy danh sách file ảnh và label thật từ thư mục test
image_files_test = [f for f in os.listdir(TEST_IMAGE_FOLDER) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
image_label_pairs_test = []
for fname in image_files_test:
    true_label = os.path.splitext(fname)[0]
    if true_label not in class_names:
        print(f"Cảnh báo: Nhãn '{true_label}' từ ảnh '{fname}' không có trong các lớp đã huấn luyện. Bỏ qua ảnh này.")
        continue
    image_label_pairs_test.append((os.path.join(TEST_IMAGE_FOLDER, fname), true_label))

print(f"Tổng số ảnh để test riêng biệt: {len(image_label_pairs_test)}")

# Chuẩn bị dữ liệu test thành các mảng numpy để đưa vào mô hình
X_test_left_eval = []
X_test_right_eval = []
X_test_pose_eval = []
y_test_encoded_eval = []

for img_path_test, true_label_test in image_label_pairs_test:
    xl, xr, xp = extract_features_from_image_multi_input(img_path_test)
    if xl is not None:
        X_test_left_eval.append(xl.flatten())
        X_test_right_eval.append(xr.flatten())
        X_test_pose_eval.append(xp.flatten())
        y_test_encoded_eval.append(true_label_test)

if not X_test_left_eval:
    print("Không có ảnh hợp lệ nào trong VSL_DATA_2 để đánh giá. Vui lòng kiểm tra thư mục và tên file.")
    exit()

X_test_left_eval = np.array(X_test_left_eval, dtype=np.float32)
X_test_right_eval = np.array(X_test_right_eval, dtype=np.float32)
X_test_pose_eval = np.array(X_test_pose_eval, dtype=np.float32)
y_test_encoded_eval = lb.transform(y_test_encoded_eval)


--- Chuẩn bị dữ liệu ảnh test từ VSL_DATA_2 để đánh giá các mô hình của Tuner ---



I0000 00:00:1752817034.181066   36663 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1752817034.290709  102570 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 570.133.07), renderer: NVIDIA GeForce RTX 3050 6GB Laptop GPU/PCIe/SSE2
I0000 00:00:1752817034.295706   36663 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
W0000 00:00:1752817034.307287  102564 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1752817034.319034  102562 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1752817034.407236  102590 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 570.133.07), renderer: NVIDIA GeForce RTX 3050 6GB Laptop GPU/PCIe/SSE2


Tổng số ảnh để test riêng biệt: 68


W0000 00:00:1752817034.458044  102589 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1752817034.482980  102578 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/home/sowndt/.local/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:359: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  y_is_multilabel = type_of_target(y).startswith("multilabel")
/home/sowndt/.local/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:528: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  y_type = type_of_target(y)



--- Đánh giá tất cả các mô hình từ Tuner trên tập ảnh VSL_DATA_2 ---

Đánh giá Trial 1/90 (Trial ID: 0051)...


/home/sowndt/.local/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


  Accuracy trên VSL_DATA_2: 41.18%
Đánh giá Trial 2/90 (Trial ID: 0050)...
  Accuracy trên VSL_DATA_2: 44.12%
Đánh giá Trial 3/90 (Trial ID: 0082)...
  Accuracy trên VSL_DATA_2: 39.71%
Đánh giá Trial 4/90 (Trial ID: 0073)...
  Accuracy trên VSL_DATA_2: 38.24%
Đánh giá Trial 5/90 (Trial ID: 0072)...
  Accuracy trên VSL_DATA_2: 44.12%
Đánh giá Trial 6/90 (Trial ID: 0083)...
  Accuracy trên VSL_DATA_2: 42.65%
Đánh giá Trial 7/90 (Trial ID: 0077)...
  Accuracy trên VSL_DATA_2: 42.65%
Đánh giá Trial 8/90 (Trial ID: 0048)...
  Accuracy trên VSL_DATA_2: 38.24%
Đánh giá Trial 9/90 (Trial ID: 0088)...
  Accuracy trên VSL_DATA_2: 42.65%
Đánh giá Trial 10/90 (Trial ID: 0046)...
  Accuracy trên VSL_DATA_2: 45.59%
Đánh giá Trial 11/90 (Trial ID: 0049)...
  Accuracy trên VSL_DATA_2: 38.24%
Đánh giá Trial 12/90 (Trial ID: 0067)...
  Accuracy trên VSL_DATA_2: 41.18%
Đánh giá Trial 13/90 (Trial ID: 0068)...
  Accuracy trên VSL_DATA_2: 39.71%
Đánh giá Trial 14/90 (Trial ID: 0069)...
  Accuracy trên VSL_


Đã chọn Trial ID 0046 làm mô hình cuối cùng.
Đã lưu mô hình cuối cùng vào file final_model_best_on_VSL_DATA2_combined_input.h5

Đã lưu LabelBinarizer vào file label_binarizer.pkl


In [11]:
# --- 5. Đánh giá tất cả các mô hình trong Tuner trên tập ảnh VSL_DATA_2 ---
print("\n--- Đánh giá tất cả các mô hình từ Tuner trên tập ảnh VSL_DATA_2 ---\n")

# SỬA ĐỔI DÒNG NÀY: Thay vì tuner.oracle.max_trials (có thể là None), sử dụng len(tuner.oracle.trials)
all_trials = tuner.oracle.get_best_trials(num_trials=len(tuner.oracle.trials)) # Lấy tất cả các trials đã chạy
trial_accuracies = []

for i, trial in enumerate(all_trials):
    print(f"Đánh giá Trial {i+1}/{len(all_trials)} (Trial ID: {trial.trial_id})...")
    try:
        # Lấy mô hình từ trial
        model_from_trial = tuner.load_model(trial)
        
        # Đánh giá mô hình trên tập VSL_DATA_2
        loss, accuracy = model_from_trial.evaluate(
            {np.concatenate((X_test_left_eval, X_test_right_eval, X_test_pose_eval), axis=1)},
            y_test_encoded_eval,
            verbose=0
        )
        trial_accuracies.append({'trial_id': trial.trial_id, 'accuracy': accuracy, 'hyperparameters': trial.hyperparameters.values})
        print(f"  Accuracy trên VSL_DATA_2: {accuracy * 100:.2f}%")
    except Exception as e:
        print(f"  Lỗi khi tải hoặc đánh giá mô hình cho Trial {trial.trial_id}: {e}")
        # Bỏ qua trial này hoặc ghi nhận accuracy là 0 nếu có lỗi

# Sắp xếp các mô hình theo accuracy trên VSL_DATA_2 (giảm dần)
trial_accuracies.sort(key=lambda x: x['accuracy'], reverse=True)

print("\n==== Kết quả đánh giá các mô hình trên VSL_DATA_2 (xếp hạng từ cao nhất) ====\n")
if not trial_accuracies:
    print("Không có mô hình nào được đánh giá thành công.")
else:
    for rank, result in enumerate(trial_accuracies):
        print(f"Hạng {rank+1}: Trial ID {result['trial_id']}")
        print(f"  Accuracy trên VSL_DATA_2: {result['accuracy'] * 100:.2f}%")
        print(f"  Siêu tham số: {result['hyperparameters']}\n")

    # Lấy mô hình có accuracy cao nhất trên VSL_DATA_2
    best_model_on_vsl_data2_info = trial_accuracies[0]
    best_model_on_vsl_data2_trial = None
    for trial in all_trials:
        if trial.trial_id == best_model_on_vsl_data2_info['trial_id']:
            best_model_on_vsl_data2_trial = trial
            break

    if best_model_on_vsl_data2_trial:
        final_best_model = tuner.load_model(best_model_on_vsl_data2_trial)
        print(f"\nĐã chọn Trial ID {best_model_on_vsl_data2_info['trial_id']} làm mô hình cuối cùng.")
        final_best_model.save('final_model_best_on_VSL_DATA2.h5')
        print(f"Đã lưu mô hình cuối cùng vào file final_model_best_on_VSL_DATA2.h5")
    else:
        print("\nKhông tìm thấy trial tương ứng cho mô hình tốt nhất để lưu.")


--- Đánh giá tất cả các mô hình từ Tuner trên tập ảnh VSL_DATA_2 ---

Đánh giá Trial 1/90 (Trial ID: 0051)...
  Lỗi khi tải hoặc đánh giá mô hình cho Trial 0051: unhashable type: 'numpy.ndarray'
Đánh giá Trial 2/90 (Trial ID: 0050)...


/home/sowndt/.local/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


  Lỗi khi tải hoặc đánh giá mô hình cho Trial 0050: unhashable type: 'numpy.ndarray'
Đánh giá Trial 3/90 (Trial ID: 0082)...
  Lỗi khi tải hoặc đánh giá mô hình cho Trial 0082: unhashable type: 'numpy.ndarray'
Đánh giá Trial 4/90 (Trial ID: 0073)...
  Lỗi khi tải hoặc đánh giá mô hình cho Trial 0073: unhashable type: 'numpy.ndarray'
Đánh giá Trial 5/90 (Trial ID: 0072)...
  Lỗi khi tải hoặc đánh giá mô hình cho Trial 0072: unhashable type: 'numpy.ndarray'
Đánh giá Trial 6/90 (Trial ID: 0083)...
  Lỗi khi tải hoặc đánh giá mô hình cho Trial 0083: unhashable type: 'numpy.ndarray'
Đánh giá Trial 7/90 (Trial ID: 0077)...
  Lỗi khi tải hoặc đánh giá mô hình cho Trial 0077: unhashable type: 'numpy.ndarray'
Đánh giá Trial 8/90 (Trial ID: 0048)...
  Lỗi khi tải hoặc đánh giá mô hình cho Trial 0048: unhashable type: 'numpy.ndarray'
Đánh giá Trial 9/90 (Trial ID: 0088)...
  Lỗi khi tải hoặc đánh giá mô hình cho Trial 0088: unhashable type: 'numpy.ndarray'
Đánh giá Trial 10/90 (Trial ID: 0046)...